In [19]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [20]:
# load data
mnist = input_data.read_data_sets("./MNIST_data", one_hot=True)

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


In [21]:
sess = tf.InteractiveSession()

In [22]:
# input and output
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
# variables
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
# initial variables
sess.run(tf.global_variables_initializer())
# linear
y = tf.matmul(x, W) + b

In [23]:
# loss
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
# back
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [24]:
# train
for _ in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run({x: batch[0], y_: batch[1]})

In [25]:
# prediction
prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
print(accuracy.eval({x: mnist.test.images, y_: mnist.test.labels}))

0.912


In [26]:
# weight and bias initialization
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.truncated_normal(shape=shape,stddev=0.1)
    return tf.Variable(initial)

In [27]:
# convolution and pooling
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding="SAME")


def max_pool_2x2(x):
    return tf.nn.max_pool(
        x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

In [28]:
# 1st convolution layer
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# reshape input
x_image = tf.reshape(x, [-1, 28, 28, 1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [29]:
# 2nd convolution layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [30]:
# densely connected layer
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [31]:
# dropout layer
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [32]:
# readout layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [33]:
# loss
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
# prediction
prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))

In [ ]:
# train

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(20000):
        batch = mnist.train.next_batch(50)
        if i % 100 == 0:
            train_accuracy = accuracy.eval({
                x: batch[0],
                y_: batch[1],
                keep_prob: 1.0
            })
            print("step %d, trainning accuracy %g" % (i, train_accuracy))
        train_step.run({x: batch[0], y_: batch[1], keep_prob: 0.5})
    print("test accuracy %g" % accuracy.eval({
        x: mnist.test.images,
        y_: mnist.test.labels,
        keep_prob: 1.0
    }))

step 0, trainning accuracy 0.08
step 100, trainning accuracy 0.88
step 200, trainning accuracy 0.94
step 300, trainning accuracy 0.8
step 400, trainning accuracy 0.96
step 500, trainning accuracy 0.96
step 600, trainning accuracy 0.9
step 700, trainning accuracy 0.94
step 800, trainning accuracy 0.98
step 900, trainning accuracy 0.94
step 1000, trainning accuracy 0.96
step 1100, trainning accuracy 0.98
step 1200, trainning accuracy 0.94
step 1300, trainning accuracy 1
step 1400, trainning accuracy 0.96
step 1500, trainning accuracy 0.96
step 1600, trainning accuracy 0.98
step 1700, trainning accuracy 0.98
step 1800, trainning accuracy 0.98
step 1900, trainning accuracy 0.98
step 2000, trainning accuracy 0.96
step 2100, trainning accuracy 0.96
step 2200, trainning accuracy 1
step 2300, trainning accuracy 1
step 2400, trainning accuracy 0.96
step 2500, trainning accuracy 1
step 2600, trainning accuracy 1
step 2700, trainning accuracy 1
step 2800, trainning accuracy 0.98
step 2900, trainn